## Dataset

In [ ]:
# Step 1: Dataset explore
# https://www.kaggle.com/tareqjoy/trainplaytennis

In [1]:
import pandas as pd
import numpy as np

In [3]:
dataset = pd.read_csv("/content/playtennis.csv")
dataset.head()

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


## Calculating the entropy of the whole dataset

In mathematics, we had to calculate the entropy of the whole dataset first like below:

    Total row = 14
    Row with "Yes" class = 9
    Row with "No" class = 5Complete entropy of dataset is -
    H(S) = - p(Yes) * log2(p(Yes)) - p(No) * log2(p(No))
        = - (9/14) * log2(9/14) - (5/14) * log2(5/14)
        = - (-0.41) - (-0.53)
        = 0.94

So, here we will do the same as the above equation.

**Method description:**
Calculates the entropy of the whole dataset.

    train_data: a pandas dataframe
    label: string, name of the label of the dataframe (=Play Tennis)
    class_list: list, unique classes of the label (=[Yes, No]).
    returns: float, calculated entropy of the whole dataframe (=0.94)



In [4]:
def calc_total_entropy(train_data, label, class_list):
    total_row = train_data.shape[0] #the total size of the dataset
    total_entropy = 0

    for c in class_list: #for each class in the label
        total_class_count = train_data[train_data[label] == c].shape[0] #number of the class
        total_class_entr = - (total_class_count/total_row)*np.log2(total_class_count/total_row) #entropy of the class
        total_entropy += total_class_entr #adding the class entropy to the total entropy of the dataset

    return total_entropy

## Calculating the entropy for the filtered dataset

Using the table of step 1, The mathematics calculation will be like below for the feature Outlook:

Categorical values of Outlook - Sunny, Overcast and RainTotal count of row containing:

    Sunny = 5
    Sunny & Yes = 2
    Sunny & No = 3>> H(Outlook=Sunny) = -(2/5)*log(2/5)-(3/5)*log(3/5) = 0.971
    
    Total count of row containing:  
    Rain = 5
    Rain & Yes = 3
    Rain & No = 2>> H(Outlook=Rain) = -(3/5)*log(3/5)-(2/5)*log(2/5) = 0.971
    
    Total count of row containing:  
    Overcast = 4
    Overcast & Yes = 4
    Overcast & No = 0>> H(Outlook=Overcast) = -(4/4)*log(4/4)-0 = 0

Note: We have to do the same for all features like Wind, Humidity etc.
The equivalent Python implementation will be like below:

Method description:

    Calculates entropy of a specific feature = value.
    feature_value_data: a pandas dataframe, which contains data that has a specific value of a feature (Ex. Only data with Outlook = Sunny)
    label: string, name of the label of the dataframe (=Play Tennis)
    class_list: list, unique classes of the label (=[Yes, No]).
    returns: float, calculated entropy of the feature value dataframe (Ex. for Outlook = Sunny, returns 0.971)

In [6]:
def calc_entropy(feature_value_data, label, class_list):
    class_count = feature_value_data.shape[0]
    entropy = 0

    for c in class_list:
        label_class_count = feature_value_data[feature_value_data[label] == c].shape[0] #row count of class c
        entropy_class = 0
        if label_class_count != 0:
            probability_class = label_class_count/class_count #probability of the class
            entropy_class = - probability_class * np.log2(probability_class)  #entropy
        entropy += entropy_class
    return entropy

## Calculating information gain for a feature

After calculating entropy, we have to calculate the information gain of that feature. In math, first, we have to calculate the information of that feature like this: (for the feature Outlook)

    I(Outlook)  = p(Sunny) * H(Outlook=Sunny) +
                  p(Rain) * H(Outlook=Rain) +
                  p(Overcast) * H(Outlook=Overcast)
                = (5/14)*0.971 + (5/14)*0.971 + (4/14)*0
                = 0.693

Then, we have to subtract this from the total entropy of the dataset which is the information gain of the feature.

    Information Gain  = H(S) - I(Outlook)
                      = 0.94 - 0.693
                      = 0.247

In python we have done like this:

**Method description:**

Calculates information gain of a feature.

    feature_name: string, the name of the feature that we want to find the information gain (Ex. Outlook)
    train_data: a pandas dataframe
    label: string, name of the label of the dataframe (=Play Tennis)
    class_list: list, unique classes of the label (=[Yes, No]).
    returns: calculated information gain of the feature (Ex. for Outlook, returns 0.247)

In [7]:
def calc_info_gain(feature_name, train_data, label, class_list):
    feature_value_list = train_data[feature_name].unique() # unqiue values of the feature
    total_row = train_data.shape[0]
    feature_info = 0.0

    for feature_value in feature_value_list:
        feature_value_data = train_data[train_data[feature_name] == feature_value] # filtering rows with that feature_value
        feature_value_count = feature_value_data.shape[0]
        feature_value_entropy = calc_entropy(feature_value_data, label, class_list) # calculcating entropy for the feature value
        feature_value_probability = feature_value_count/total_row
        feature_info += feature_value_probability * feature_value_entropy # calculating information of the feature value

    return calc_total_entropy(train_data, label, class_list) - feature_info # calculating information gain by subtracting

## Finding the most informative feature (feature with highest information gain)

Like Outlook feature, We have to calculate information gain for every feature in the dataset. Then we have to select the feature with the highest information gain. After calculating mathematically we will find the values like below:

Information gain:

    Outlook = 0.247 (Highest value)
    Temperature = 0.0292
    Humidity = 0.153
    Wind = 0.048

As the feature Outlook has the highest value, so it will be selected for our tree node.

**Method description:**

    Finds the most informative feature from the current dataset.
    train_data: a pandas dataframe
    label: string, name of the label of the dataframe (=Play Tennis)
    class_list: list, unique classes of the label (=[Yes, No]).
    returns: string, the feature name

In [8]:
def find_most_informative_feature(train_data, label, class_list):
    feature_list = train_data.columns.drop(label) # finding the feature names in the dataset
                                                  # N.B. label is not a feature, so dropping it
    max_info_gain = -1
    max_info_feature = None

    for feature in feature_list: # for each feature in the dataset
        feature_info_gain = calc_info_gain(feature, train_data, label, class_list)
        if max_info_gain < feature_info_gain: # selecting feature name with highest information gain
            max_info_gain = feature_info_gain
            max_info_feature = feature

    return max_info_feature

## Adding a node to the tree

As we have found the feature name with the highest information gain, we have to generate a node in the tree and its value as a branch. For example, we have selected **Outlook**, so we have to add **Outlook as a node in the tree** and its value **Sunny or Rain or Overcast as a branch**.

Outlook is selected as Node.

    (Outlook = Sunny): Not pure class, contains both class Yes and No
    (Outlook = Overcast): Pure class, contains only one class Yes
    (Outlook = Rain): Not pure class, contains both class Yes and No


After selecting a pure class, we have to remove the rows from the dataset corresponding to the feature value. **Ex: we have to remove rows with Outlook = Overcast.** The resultant dataset will be updated.


Method description:

    Generates subtree of a feature and removes the feature = value from the dataset. The tree might contain ‘?’ as a value if the tree node isn’t a pure class.
    feature_name: string, the name of the feature that we want to add to tree and shrink dataset (Ex. Outlook)
    train_data: a pandas dataframe
    label: string, name of the label of the dataframe (=Play Tennis)
    class_list: list, unique classes of the label (=[Yes, No]).
    returns: tuple (dictionary, dataframe), the tree node with it’s branches and the updated dataset

In [9]:
def generate_sub_tree(feature_name, train_data, label, class_list):
    feature_value_count_dict = train_data[feature_name].value_counts(sort=False) #dictionary of the count of unqiue feature value
    tree = {} #sub tree or node

    for feature_value, count in feature_value_count_dict.iteritems():
        feature_value_data = train_data[train_data[feature_name] == feature_value] #dataset with only feature_name = feature_value

        assigned_to_node = False #flag for tracking feature_value is pure class or not
        for c in class_list: #for each class
            class_count = feature_value_data[feature_value_data[label] == c].shape[0] #count of class c

            if class_count == count: #count of (feature_value = count) of class (pure class)
                tree[feature_value] = c #adding node to the tree
                train_data = train_data[train_data[feature_name] != feature_value] #removing rows with feature_value
                assigned_to_node = True
        if not assigned_to_node: #not pure class
            tree[feature_value] = "?" #as feature_value is not a pure class, it should be expanded further,
                                      #so the branch is marking with ?

    return tree, train_data

##  Performing ID3 Algorithm and generating Tree

The overall step is:

    Finding the most informative feature
    Making a tree node with a feature name and feature values as branches
    - If pure class, adding leaf node (= Class) to the tree node
    - If impure class, adding an expandable node (= ‘?’) to the tree node
    Shrinking/Updating the dataset according to the pure class
    Adding the node with branches into a tree
    Expand the branch of the next impure class (= ‘?’) with an updated dataset

The recursion endpoint:

    The dataset becomes empty after updating
    There is no expandable branch (= all pure class)

**Method description:**

Generates a tree using dictionary of dictionaries. The leaf node of the tree would be value of a feature = class name. The resultant tree is returned by reference.

    root: dictionary, which will contain the resultant tree through recursive subtree, initially it should be an empty dictionary, after the recursive calls it should contain the result
    prev_feature_value: Any datatype (Int or Float or String etc.) depending on the datatype of the previous feature, the previous value of the pointed node/feature, initially it should be None
    train_data: a pandas dataframe
    label: string, name of the label of the dataframe (=Play Tennis)
    class_list: list, unique classes of the label (=[Yes, No])
    returns: None

In [10]:
def make_tree(root, prev_feature_value, train_data, label, class_list):
    if train_data.shape[0] != 0: #if dataset becomes enpty after updating
        max_info_feature = find_most_informative_feature(train_data, label, class_list) #most informative feature
        tree, train_data = generate_sub_tree(max_info_feature, train_data, label, class_list) #getting tree node and updated dataset
        next_root = None

        if prev_feature_value != None: #add to intermediate node of the tree
            root[prev_feature_value] = dict()
            root[prev_feature_value][max_info_feature] = tree
            next_root = root[prev_feature_value][max_info_feature]
        else: #add to root of the tree
            root[max_info_feature] = tree
            next_root = root[max_info_feature]

        for node, branch in list(next_root.items()): #iterating the tree node
            if branch == "?": #if it is expandable
                feature_value_data = train_data[train_data[max_info_feature] == node] #using the updated dataset
                make_tree(next_root, node, feature_value_data, label, class_list) #recursive call with updated dataset

## Finding unique classes of the label and Starting the algorithm

We can start calling the recursive tree building algorithm of ID3 after finding the class names.

**Method description:** Generates id3 tree.
    
    train_data_m: a pandas dataframe
    label: string, name of the label of the dataframe (=Play Tennis)
    returns: (nested) dictionary, the decision tree

In [11]:
def id3(train_data_m, label):
    train_data = train_data_m.copy() #getting a copy of the dataset
    tree = {} #tree which will be updated
    class_list = train_data[label].unique() #getting unqiue classes of the label
    make_tree(tree, None, train_data, label, class_list) #start calling recursion
    return tree

In [12]:
tree = id3(dataset, 'Play Tennis')

<ipython-input-9-b182bf0ec406>:5: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for feature_value, count in feature_value_count_dict.iteritems():
<ipython-input-9-b182bf0ec406>:5: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for feature_value, count in feature_value_count_dict.iteritems():
<ipython-input-9-b182bf0ec406>:5: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for feature_value, count in feature_value_count_dict.iteritems():


In [13]:
tree

{'Outlook': {'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}},
  'Overcast': 'Yes',
  'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}}}

In [25]:
import pydot

def draw(parent_name, child_name):
    edge = pydot.Edge(parent_name, child_name)
    graph.add_edge(edge)

def visit(node, parent=None):
    for k,v in node.items():# If using python3, use node.items() instead of node.iteritems()
        if isinstance(v, dict):
            # We start with the root node whose parent is None
            # we don't want to graph the None node
            if parent:
                draw(parent, k)
            visit(v, k)
        else:
            draw(parent, k)
            # drawing the label using a distinct name
            draw(k, k+'_'+v)

graph = pydot.Dot(graph_type='graph')
visit(tree)
graph.write_png('decision_tree.png')

## Predicting from the tree

We have generated the tree. Now, we can use the tree for prediction. We will recursively traverse the nested dictionary until any leaf node (= class) is found. Remember that, the key of the dictionary is feature name which is the datatype of string, and the value is either a dictionary which means we have to traverse the tree more or any basic datatype like string or int or float depending on the class data type, which means it’s a leaf node.

If we want to predict Outlook = `Rain` and Wind = `Strong` we have to traverse the dictionary like that:

In [14]:
def predict(tree, instance):
    if not isinstance(tree, dict): #if it is leaf node
        return tree #return the value
    else:
        root_node = next(iter(tree)) #getting first key/feature name of the dictionary
        feature_value = instance[root_node] #value of the feature
        if feature_value in tree[root_node]: #checking the feature value in current tree node
            return predict(tree[root_node][feature_value], instance) #goto next feature
        else:
            return None

## Evaluation

To evaluate the model i.e. the tree we need a labeled test dataset. Then after predicting we can calculate the difference of actual and predicted value in percentage.

**Method description:**     Evaluates the accuracy of a id3 tree by testing against the expected result

    tree: dictionary (of dictionaries), a decision tree
    test_data_m: a pandas dataframe/test dataset
    returns: float, the accuracy of the tree

In [19]:
def evaluate(tree, test_data_m, label):
    correct_preditct = 0
    wrong_preditct = 0
    for index, row in test_data_m.iterrows(): #for each row in the dataset
        result = predict(tree, test_data_m.iloc[index]) #predict the row
        if result == test_data_m[label].iloc[index]: #predicted value and expected value is same or not
            correct_preditct += 1 #increase correct count
        else:
            wrong_preditct += 1 #increase incorrect count
    accuracy = correct_preditct / (correct_preditct + wrong_preditct) #calculating accuracy
    return accuracy

In [18]:
accuracy = evaluate(tree, dataset, 'Play Tennis') #evaluating the test dataset
accuracy

1.0